In [ ]:
%pip install psycopg2-binary voyageai tqdm

In [2]:
import psycopg2
from voyageai import Client

from tqdm import tqdm

In [ ]:

client = Client(
    api_key="pa-_7vCVNq7lM6mfe-E8cEyOkEyVB0soI3HwHwlO2BJwW0"
)
# embeddings = client.embed("This is sample message", model="voyage-law-2")
# print(embeddings)
CONN_STRING = "dbname=CaseGraph3 user=postgres1dev password=dev4023TcupSoda host=legawritesql.postgres.database.azure.com"

def GetEmbedding(text, model="voyage-law-2"):
    client = Client(
        api_key="pa-_7vCVNq7lM6mfe-E8cEyOkEyVB0soI3HwHwlO2BJwW0"
    )
    embedding = client.embed(text, model=model)
    embedding = embedding.embeddings[0]
    return embedding

def GetEmbeddingsBatch(texts, model="voyage-law-2"):
    client = Client(
        api_key="pa-_7vCVNq7lM6mfe-E8cEyOkEyVB0soI3HwHwlO2BJwW0"
    )
    embeddings = client.embed(texts, model=model)
    embeddings = embeddings.embeddings
    return embeddings


In [4]:
def CreateEmbeddings(name_of_target_table, name_of_target_column, name_of_embedding_column):
    conn = psycopg2.connect(CONN_STRING)
    cursor = conn.cursor()

    check_column_query = """
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name=%s AND column_name=%s;
    """
    cursor.execute(check_column_query, (name_of_target_table, name_of_embedding_column))
    
    if cursor.fetchone() is None:
        alter_table_query = f'ALTER TABLE public.{name_of_target_table} ADD COLUMN {name_of_embedding_column} float[];'
        cursor.execute(alter_table_query)
        conn.commit()
    else:
        print("Err condition")
    
    select_query = f"SELECT principle_id, {name_of_target_column} FROM public.{name_of_target_table};"
    cursor.execute(select_query)
    rows = cursor.fetchall()
    for row in tqdm(rows, position=0, leave=True, total=len(rows), desc=f"Creating embeddings for {name_of_target_table}, Items"):
        id, text = row
        embedding = GetEmbedding(text)
        update_query = f"UPDATE public.{name_of_target_table} SET {name_of_embedding_column} = (%s) WHERE principle_id = {id};"
        cursor.execute(update_query, (embedding,))
        conn.commit()

    cursor.close()
    conn.close()
    
    print("Embeddings created and updated successfully.")
    
def CreateEmbeddingsBatch(name_of_target_table, name_of_target_column, name_of_embedding_column, batch_size=100):
    conn = psycopg2.connect(CONN_STRING)
    cursor = conn.cursor()

    check_column_query = """
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name=%s AND column_name=%s;
    """
    cursor.execute(check_column_query, (name_of_target_table, name_of_embedding_column))
    if cursor.fetchone() is None:
        alter_table_query = f'ALTER TABLE public.{name_of_target_table} ADD COLUMN {name_of_embedding_column} float[];'
        cursor.execute(alter_table_query)
        conn.commit()

    select_query = f"SELECT domain_id, {name_of_target_column} FROM public.{name_of_target_table};"
    cursor.execute(select_query)
    rows = cursor.fetchall()

    total_rows = len(rows)
    # print(f"Processing {total_rows} rows in batches of {batch_size}...")

    for batch_start in tqdm(range(0, total_rows, batch_size), desc=f"Creating embeddings for {name_of_target_table}, batches", position=0, leave=True):
        batch_rows = rows[batch_start:batch_start + batch_size]
        ids = [row[0] for row in batch_rows]
        texts = [row[1] for row in batch_rows]

        embeddings = GetEmbeddingsBatch(texts)

        update_data = list(zip(embeddings, ids))
        #update_query = f"UPDATE public.{name_of_target_table} SET {name_of_embedding_column} = %s WHERE domain_id = %s;"
        update_query = f"""UPDATE public.{name_of_target_table} SET {name_of_embedding_column} = %s::vector WHERE domain_id = %s;"""
        

        cursor.executemany(update_query, update_data)
        conn.commit()

    cursor.close()
    conn.close()

    print("Embeddings created and updated successfully.")

In [ ]:
# Create embeddings for a single column in a table item by item
# table: defenses, column: name, embedding_column: name_embedding
CreateEmbeddings(
    name_of_target_table="legal_principles",
    name_of_target_column="type || ',' || context",
    name_of_embedding_column="name_embedding"
)

In [12]:
def CreateEmbeddingsBatch_LP(name_of_target_table, name_of_target_column, name_of_embedding_column, batch_size=100):
    conn = psycopg2.connect(CONN_STRING)
    cursor = conn.cursor()

    check_column_query = """
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name=%s AND column_name=%s;
    """
    cursor.execute(check_column_query, (name_of_target_table, name_of_embedding_column))
    if cursor.fetchone() is None:
        alter_table_query = f'ALTER TABLE public.{name_of_target_table} ADD COLUMN {name_of_embedding_column} vector;'
        cursor.execute(alter_table_query)
        conn.commit()

    select_query = f"SELECT principle_id, {name_of_target_column} FROM public.{name_of_target_table};"
    cursor.execute(select_query)
    rows = cursor.fetchall()

    total_rows = len(rows)
    # print(f"Processing {total_rows} rows in batches of {batch_size}...")

    for batch_start in tqdm(range(0, total_rows, batch_size), desc=f"Creating embeddings for {name_of_target_table}, batches", position=0, leave=True):
        batch_rows = rows[batch_start:batch_start + batch_size]
        ids = [row[0] for row in batch_rows]
        texts = [row[1] for row in batch_rows]

        embeddings = GetEmbeddingsBatch(texts)

        update_data = list(zip(embeddings, ids))
        #update_query = f"UPDATE public.{name_of_target_table} SET {name_of_embedding_column} = %s WHERE domain_id = %s;"
        update_query = f"""UPDATE public.{name_of_target_table} SET {name_of_embedding_column} = %s::vector WHERE principle_id = %s;"""
        

        cursor.executemany(update_query, update_data)
        conn.commit()

    cursor.close()
    conn.close()

    print("Embeddings created and updated successfully.")
    
def CreateEmbeddingsBatch_issue(name_of_target_table, name_of_target_column, name_of_embedding_column, batch_size=100):
    conn = psycopg2.connect(CONN_STRING)
    cursor = conn.cursor()

    check_column_query = """
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name=%s AND column_name=%s;
    """
    cursor.execute(check_column_query, (name_of_target_table, name_of_embedding_column))
    if cursor.fetchone() is None:
        alter_table_query = f'ALTER TABLE public.{name_of_target_table} ADD COLUMN {name_of_embedding_column} vector;'
        cursor.execute(alter_table_query)
        conn.commit()

    select_query = f"SELECT issue_id, {name_of_target_column} FROM public.{name_of_target_table};"
    cursor.execute(select_query)
    rows = cursor.fetchall()

    total_rows = len(rows)
    # print(f"Processing {total_rows} rows in batches of {batch_size}...")

    for batch_start in tqdm(range(0, total_rows, batch_size), desc=f"Creating embeddings for {name_of_target_table}, batches", position=0, leave=True):
        batch_rows = rows[batch_start:batch_start + batch_size]
        ids = [row[0] for row in batch_rows]
        texts = [row[1] for row in batch_rows]

        embeddings = GetEmbeddingsBatch(texts)

        update_data = list(zip(embeddings, ids))
        #update_query = f"UPDATE public.{name_of_target_table} SET {name_of_embedding_column} = %s WHERE domain_id = %s;"
        update_query = f"""UPDATE public.{name_of_target_table} SET {name_of_embedding_column} = %s::vector WHERE issue_id = %s;"""
        

        cursor.executemany(update_query, update_data)
        conn.commit()

    cursor.close()
    conn.close()

    print("Embeddings created and updated successfully.")

def CreateEmbeddingsBatch_COA(name_of_target_table, name_of_target_column, name_of_embedding_column, batch_size=100):
    conn = psycopg2.connect(CONN_STRING)
    cursor = conn.cursor()

    check_column_query = """
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name=%s AND column_name=%s;
    """
    cursor.execute(check_column_query, (name_of_target_table, name_of_embedding_column))
    if cursor.fetchone() is None:
        alter_table_query = f'ALTER TABLE public.{name_of_target_table} ADD COLUMN {name_of_embedding_column} vector;'
        cursor.execute(alter_table_query)
        conn.commit()

    select_query = f"SELECT cause_id, {name_of_target_column} FROM public.{name_of_target_table};"
    cursor.execute(select_query)
    rows = cursor.fetchall()

    total_rows = len(rows)
    # print(f"Processing {total_rows} rows in batches of {batch_size}...")

    for batch_start in tqdm(range(0, total_rows, batch_size), desc=f"Creating embeddings for {name_of_target_table}, batches", position=0, leave=True):
        batch_rows = rows[batch_start:batch_start + batch_size]
        ids = [row[0] for row in batch_rows]
        texts = [row[1] for row in batch_rows]

        embeddings = GetEmbeddingsBatch(texts)

        update_data = list(zip(embeddings, ids))
        #update_query = f"UPDATE public.{name_of_target_table} SET {name_of_embedding_column} = %s WHERE domain_id = %s;"
        update_query = f"""UPDATE public.{name_of_target_table} SET {name_of_embedding_column} = %s::vector WHERE cause_id = %s;"""
        

        cursor.executemany(update_query, update_data)
        conn.commit()

    cursor.close()
    conn.close()

    print("Embeddings created and updated successfully.")
    
def CreateEmbeddingsBatch_rule(name_of_target_table, name_of_target_column, name_of_embedding_column, batch_size=100):
    conn = psycopg2.connect(CONN_STRING)
    cursor = conn.cursor()

    check_column_query = """
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name=%s AND column_name=%s;
    """
    cursor.execute(check_column_query, (name_of_target_table, name_of_embedding_column))
    if cursor.fetchone() is None:
        alter_table_query = f'ALTER TABLE public.{name_of_target_table} ADD COLUMN {name_of_embedding_column} vector;'
        cursor.execute(alter_table_query)
        conn.commit()

    select_query = f"SELECT ruling_id, {name_of_target_column} FROM public.{name_of_target_table};"
    cursor.execute(select_query)
    rows = cursor.fetchall()

    total_rows = len(rows)
    # print(f"Processing {total_rows} rows in batches of {batch_size}...")

    for batch_start in tqdm(range(0, total_rows, batch_size), desc=f"Creating embeddings for {name_of_target_table}, batches", position=0, leave=True):
        batch_rows = rows[batch_start:batch_start + batch_size]
        ids = [row[0] for row in batch_rows]
        texts = [row[1] for row in batch_rows]

        embeddings = GetEmbeddingsBatch(texts)

        update_data = list(zip(embeddings, ids))
        #update_query = f"UPDATE public.{name_of_target_table} SET {name_of_embedding_column} = %s WHERE domain_id = %s;"
        update_query = f"""UPDATE public.{name_of_target_table} SET {name_of_embedding_column} = %s::vector WHERE ruling_id = %s;"""
        

        cursor.executemany(update_query, update_data)
        conn.commit()

    cursor.close()
    conn.close()

    print("Embeddings created and updated successfully.")

In [ ]:
# Create embeddings (x18 faster with 100 batch size, 128 max)
# table: defenses, column: name, embedding_column: name_embedding

CreateEmbeddings(
    name_of_target_table="legal_principles",
    name_of_target_column="type || ',' || context",
    name_of_embedding_column="name_embedding"
)


Creating embeddings for legal_principles, batches: 100%|██████████| 110/110 [02:48<00:00,  1.53s/it]

Embeddings created and updated successfully.


In [19]:
# CreateEmbeddingsBatch_LP(
#     name_of_target_table="legal_principles",
#     name_of_target_column="doctrine_principle",
#     name_of_embedding_column="name_embedding",
#     batch_size=128

# )
# CreateEmbeddingsBatch_LP(
#     name_of_target_table="legal_principles",
#     name_of_target_column="doctrine_principle",
#     name_of_embedding_column="name_embedding",
#     batch_size=128

# )
CreateEmbeddingsBatch_rule(
    name_of_target_table="case_rulings",
    name_of_target_column="legal_principle_text",
    name_of_embedding_column="ruling_embedding",
    batch_size=128
)
# CreateEmbeddingsBatch_issue(
#     name_of_target_table="legal_principles",
#     name_of_target_column="issue_text",
#     name_of_embedding_column="issues_embedding",
#     batch_size=128
# )
# CreateEmbeddingsBatch_COA(
#     name_of_target_table="causes_of_action",
#     name_of_target_column="name",
#     name_of_embedding_column="name_embedding",
#     batch_size=128
# )

Creating embeddings for case_rulings, batches: 0it [00:00, ?it/s]

Embeddings created and updated successfully.


In [5]:
CreateEmbeddingsBatch(
    name_of_target_table="law_domains",
    name_of_target_column=(
        "coalesce(broad, '')"
        " || '->' || coalesce(subdomain, '')"
        " || '->' || coalesce(\"specific\", '')"
    ),
    name_of_embedding_column="domain_embedding",
    batch_size=128
)

Creating embeddings for law_domains, batches: 100%|██████████| 532/532 [12:25<00:00,  1.40s/it]

Embeddings created and updated successfully.


In [ ]:
CreateEmbeddingsBatch(
    name_of_target_table="law_domains",
    name_of_target_column=(
        "coalesce(broad, '') || '->' || coalesce(subdomain, '')"
    ),
    name_of_embedding_column="broad_subdomain_embedding",
    batch_size=128
)

In [ ]:
emb_LD = GetEmbedding("Employment Law -> Discrimination -> Retaliation Protection")
emb_LP = GetEmbedding("Witness Protection Principle. FEHA protects employees who participate in workplace discrimination investigations.")


conn = psycopg2.connect(CONN_STRING)
cursor = conn.cursor()

query = """
SELECT DISTINCT 
    c.case_id,
    c."name",
    d.domain_id,
    d.broad,
    d.subdomain,
    d.specific,
    lp.name AS principle_name,
    d.domain_embedding <=> %s::vector AS distanceLD,
    lp.name_embedding <=> %s::vector AS distanceLP,
    (
      0.4 * (d.domain_embedding <=> %s::vector) +
      0.6 * (lp.name_embedding <=> %s::vector)
    ) AS combined_distance
FROM law_domains d
INNER JOIN case_law_domains cld ON cld.domain_id = d.domain_id
INNER JOIN cases c ON c.case_id::text = cld.case_id::text
INNER JOIN legal_principles lp ON lp.case_id = c.case_id
WHERE 
    d.domain_embedding IS NOT NULL
    AND lp.name_embedding IS NOT NULL
ORDER BY combined_distance ASC
LIMIT 200;
"""

cursor.execute(query, (emb_LD, emb_LP, emb_LD, emb_LP))
results = cursor.fetchall()

for row in results:
    print(row)

cursor.close()
conn.close()

In [ ]:
emb_LP = GetEmbedding("Witness Protection Principle. FEHA protects employees who participate in workplace discrimination investigations.")



conn = psycopg2.connect(CONN_STRING)
cursor = conn.cursor()

query = """
SELECT DISTINCT 
    c.case_id,
    c."name",
    lp.name AS principle_name,
    lp.name_embedding <=> %s::vector AS distanceLP
FROM legal_principles lp
INNER JOIN cases c 
    ON lp.case_id = c.case_id
INNER JOIN case_law_domains cld 
    ON cld.case_id=c.case_id
INNER JOIN law_domains d    
    on cld.domain_id = d.domain_id    
WHERE lp.name_embedding IS NOT NULL and d.domain_embedding IS NOT NULL
and d.
ORDER BY distanceLP ASC
LIMIT 200;
"""
cursor.execute(query, (emb_LP,))
results = cursor.fetchall()

for row in results:
    print(row)

cursor.close()
conn.close()

In [ ]:
emb_domain = GetEmbedding("Employment Law -> Discrimination")
emb_principle = GetEmbedding("Witness Protection Principle. FEHA protects employees who participate in workplace discrimination investigations.")


conn = psycopg2.connect(CONN_STRING)
cursor = conn.cursor()

query = """
SELECT DISTINCT 
    c.case_id,
    c."name",
    d.broad,
    d.subdomain,
    d.specific,
    lp.name AS principle_name,
    d.broad_subdomain_embedding <=> %s::vector AS distance_domain,
    lp.name_embedding <=> %s::vector AS distance_principle,
    (
      0.7 * (d.broad_subdomain_embedding <=> %s::vector) +
      0.3 * (lp.name_embedding <=> %s::vector)
    ) AS combined_distance
FROM law_domains d
INNER JOIN case_law_domains cld ON cld.domain_id = d.domain_id
INNER JOIN cases c ON c.case_id::text = cld.case_id::text
INNER JOIN legal_principles lp ON lp.case_id = c.case_id
WHERE 
    d.broad_subdomain_embedding IS NOT NULL
    AND lp.name_embedding IS NOT NULL
ORDER BY combined_distance ASC
LIMIT 200;

"""
cursor.execute(query, (emb_domain, emb_principle, emb_domain, emb_principle))
results = cursor.fetchall()

for row in results:
    print(row)

cursor.close()
conn.close()

In [36]:
import psycopg2
CONN_STRING = "dbname=KG-Extractor user=postgres1dev password=dev4023TcupSoda host=legawritesql.postgres.database.azure.com"

conn = psycopg2.connect(CONN_STRING)
case_id = 'steele-v-youthful-offender-parole-bd'

query = """
    UPDATE cases
    SET text = %s
    WHERE case_id = %s
"""

with open("steele-v-youthful-offender-parole-bd.txt", "r", encoding="utf-8") as file:
    large_text = file.read()

try:
    with conn:
        with conn.cursor() as cursor:
            cursor.execute(query, (large_text, case_id))  
            print("Update successful.")
except Exception as e:
    print("Error:", e)
finally:
    conn.close()


Update successful.


In [ ]:
print("""\n  <NameSelectedCOA>Unpaid Missed Meal Breaks</NameSelectedCOA>\n  <ProceduralDefenses>\n    <ProceduralDefense1>\n      <ProceduralDefense_name>Failure to State Essential Elements</ProceduralDefense_name>\n      <LegalPrinciple>Complaint must specify timing and nature of meal violations</LegalPrinciple>\n      <LegalPrinciple_type>Pleading Requirement</LegalPrinciple_type>\n      <applicability>Complaint lacks specific meal period violation details</applicability>\n      <LawDomain>\n        Labor Law -> Wage and Hour -> Meal Break Requirements\n      </LawDomain>\n      <casesCited>\n        <caseCited_1>\n          <caseCited_1_name>Brinker Restaurant Corp. v. Superior Court (2012)</caseCited_1_name>\n          <caseCited_1_applicability>Establishes specific pleading requirements for meal break claims</caseCited_1_applicability>\n          <WhyCaseIsThisRelevant>Defines elements required in meal break violation complaints</WhyCaseIsThisRelevant>\n          <CaseEstablishesRule>Complaint must allege specific instances of meal violations</CaseEstablishesRule>\n        </caseCited_1>\n        <caseCited_2>\n          <caseCited_2_name>Lamps Plus Overtime Cases (2015)</caseCited_2_name>\n          <caseCited_2_applicability>Requires specific factual allegations for each violation</caseCited_2_applicability>\n          <WhyCaseIsThisRelevant>Sets pleading standard for meal break claims</WhyCaseIsThisRelevant>\n          <CaseEstablishesRule>General allegations insufficient without specific violation details</CaseEstablishesRule>\n        </caseCited_2>\n      </casesCited>\n      <AppliedArgument>Complaint only states general allegations without specific instances</AppliedArgument>\n      <AppliedArgument_type>Insufficient Pleading Detail</AppliedArgument_type>\n    </ProceduralDefense1>\n  </ProceduralDefenses>\n  <SubstantiveDefenses>\n    <SubstantiveDefense1>\n      <SubstantiveDefense_name>Exempt Employee Classification Defense</SubstantiveDefense_name>\n      <LegalPrinciple>Exempt employees not entitled to meal break requirements</LegalPrinciple>\n      <LegalPrinciple_type>Employment Classification Law</LegalPrinciple_type>\n      <applicability>Initial exempt classification bars meal break claims</applicability>\n      <LawDomain>\n        Employment Law -> Employee Classification -> Exempt Status\n      </LawDomain>\n      <casesCited>\n        <caseCited_1>\n          <caseCited_1_name>Sav-On Drug Stores v. Superior Court (2004)</caseCited_1_name>\n          <caseCited_1_applicability>Exempt status determined at time of classification</caseCited_1_applicability>\n          <WhyCaseIsThisRelevant>Addresses classification impact on meal break rights</WhyCaseIsThisRelevant>\n          <CaseEstablishesRule>Initial exempt classification controls unless formally changed</CaseEstablishesRule>\n        </caseCited_1>\n      </casesCited>\n      <AppliedArgument>Initial exempt classification precludes meal break claims</AppliedArgument>\n      <AppliedArgument_type>Classification Based Defense</AppliedArgument_type>\n    </SubstantiveDefense1>\n  </SubstantiveDefenses>\n""")

In [ ]:
test_xml = """
    <NameSelectedCOA>Wrongful Termination in Violation of Public Policy</NameSelectedCOA>
  
    <ProceduralDefenses>
        <ProceduralDefense1>
            <ProceduralDefense_name>Failure to Exhaust Administrative Remedies</ProceduralDefense_name>
            <LegalPrinciple>Administrative procedures must be completed before court action</LegalPrinciple>
            <LegalPrinciple_type>Administrative Law Requirement</LegalPrinciple_type>
            <applicability>Labor violations require prior agency review</applicability>
            <LawDomain>Employment Law -> Administrative Procedures -> Exhaustion Doctrine</LawDomain>
            <casesCited>
                <caseCited_1>
                    <caseCited_1_name>Campbell v. Regents of University of California (2005)</caseCited_1_name>
                    <caseCited_1_applicability>Establishes mandatory exhaustion requirement for employment claims</caseCited_1_applicability>
                    <WhyCaseIsThisRelevant>Requires administrative process before judicial review</WhyCaseIsThisRelevant>
                    <CaseEstablishesRule>Administrative remedies must be exhausted before court action</CaseEstablishesRule>
                </caseCited_1>
            </casesCited>
            <AppliedArgument>Complaint lacks allegations of administrative procedure completion</AppliedArgument>
            <AppliedArgument_type>Procedural Prerequisites Not Met</AppliedArgument_type>
        </ProceduralDefense1>
    </ProceduralDefenses>

    <SubstantiveDefenses>
        <SubstantiveDefense1>
            <ProceduralDefense_name>Insufficient Public Policy Nexus</ProceduralDefense_name>
            <LegalPrinciple>Public policy must be fundamental and substantial</LegalPrinciple>
            <LegalPrinciple_type>Public Policy Requirements</LegalPrinciple_type>
            <applicability>Policy violation must be clearly established</applicability>
            <LawDomain>Employment Law -> Public Policy -> Wrongful Termination</LawDomain>
            <casesCited>
                <caseCited_1>
                    <caseCited_1_name>Gantt v. Sentry Insurance (1992)</caseCited_1_name>
                    <caseCited_1_applicability>Defines requirements for public policy claims</caseCited_1_applicability>
                    <WhyCaseIsThisRelevant>Sets standards for public policy wrongful termination</WhyCaseIsThisRelevant>
                    <CaseEstablishesRule>Policy must be fundamental, substantial, well-established</CaseEstablishesRule>
                </caseCited_1>
            </casesCited>
            <AppliedArgument>Complaint fails to specify clear public policy source</AppliedArgument>
            <AppliedArgument_type>Policy Source Not Identified</AppliedArgument_type>
        </SubstantiveDefense1>

        <SubstantiveDefense2>
            <ProceduralDefense_name>Legitimate Business Reason Defense</ProceduralDefense_name>
            <LegalPrinciple>Employment changes permitted for business operations</LegalPrinciple>
            <LegalPrinciple_type>Business Judgment Rule</LegalPrinciple_type>
            <applicability>Role changes within employer discretion</applicability>
            <LawDomain>Employment Law -> At-Will Employment -> Business Necessity</LawDomain>
            <casesCited>
                <caseCited_1>
                    <caseCited_1_name>Guz v. Bechtel National Inc. (2000)</caseCited_1_name>
                    <caseCited_1_applicability>Supports employer discretion in business decisions</caseCited_1_applicability>
                    <WhyCaseIsThisRelevant>Establishes business judgment protection</WhyCaseIsThisRelevant>
                    <CaseEstablishesRule>Employers have discretion in operational decisions</CaseEstablishesRule>
                </caseCited_1>
            </casesCited>
            <AppliedArgument>Role changes reflect legitimate business reorganization</AppliedArgument>
            <AppliedArgument_type>Business Decision Protection</AppliedArgument_type>
        </SubstantiveDefense2>
    </SubstantiveDefenses>
"""

In [ ]:
import re

def xml_to_dict(xml_string):
    xml_string = xml_string.strip()
    tag_pattern = re.compile(r'<(\w+)>(.*?)</\1>', re.DOTALL)
    
    def parse(fragment):
        fragment = fragment.strip()
        if not tag_pattern.search(fragment):
            return fragment
        
        result = {}
        for match in tag_pattern.finditer(fragment):
            tag = match.group(1)
            content = match.group(2).strip()
            parsed_content = parse(content)
            
            if tag in result:
                if isinstance(result[tag], list):
                    result[tag].append(parsed_content)
                else:
                    result[tag] = [result[tag], parsed_content]
            else:
                result[tag] = parsed_content
        return result
    
    return parse(xml_string)

parsed_xml = xml_to_dict(test_xml)
parsed_xml

In [ ]:
def GetCase(input_embedding_coa_name_string, input_embedding_procedural_defense_name_string, input_legal_principal_string):
    input_embedding_coa_name = GetEmbedding(input_embedding_coa_name_string)
    input_embedding_procedural_defense_name = GetEmbedding(input_embedding_procedural_defense_name_string)
    input_legal_principal = GetEmbedding(input_legal_principal_string)

    query = f"""
        SELECT f.name, f.case_law, f.defence_name, f.Distance1, f.Distance2, f.Distance3
            FROM (
                SELECT c.name, d.case_law, d.name as defence_name, d.legal_principle_embedding,
                    cosine_similarity(%s::double precision[], c.name_embedding) as Distance1,
                    cosine_similarity(%s::double precision[], d.name_embedding) as Distance2,
                    cosine_similarity(%s::double precision[], d.legal_principle_embedding) as Distance3
                FROM public.causes_of_action AS c
                INNER JOIN defenses d
                    on c.id=d.cause_of_action_id
            ) AS f
        WHERE f.Distance1 >= 0.4 and f.Distance2 >= 0.4
        ORDER BY f.Distance1 DESC, f.Distance2 DESC, f.Distance3 DESC
    """

    conn = psycopg2.connect(CONN_STRING)
    cursor = conn.cursor()

    cursor.execute(query, (input_embedding_coa_name, input_embedding_procedural_defense_name, input_legal_principal))
    cosine_similarity = cursor.fetchall()
    cursor.close()
    conn.close()
    return cosine_similarity[0]

In [ ]:
name_of_selected_coa = parsed_xml["NameSelectedCOA"]
# for procedural_defense in parsed_xml["ProceduralDefenses"].values():
#     name_of_procedural_defense = procedural_defense["ProceduralDefense_name"]
#     # legal_principle = procedural_defense["LegalPrinciple"]
#     # print(name_of_procedural_defense)
#     print(name_of_selected_coa)
#     print(name_of_procedural_defense)
#     top_case = GetCase(name_of_selected_coa, name_of_procedural_defense)
#     print(top_case)
#     procedural_defense["casesFound"] = top_case

for substantive_defense in parsed_xml["SubstantiveDefenses"].values():
    name_of_substantive_defense = substantive_defense["ProceduralDefense_name"]
    legal_principle = substantive_defense["LegalPrinciple"]
    # print(name_of_substantive_defense)
    print(name_of_selected_coa)
    print(name_of_substantive_defense)
    top_case = GetCase(name_of_selected_coa, name_of_substantive_defense, legal_principle)
    case_name = top_case[1]
    print()
    print(case_name)
    print(top_case)
    substantive_defense["casesFound"] = top_case[1]
    

parsed_xml

In [ ]:
input_embedding_coa_name = client.embed("Wrongful Termination in Violation of Public Policy", model="voyage-law-2")
input_embedding_coa_name = input_embedding_coa_name.embeddings[0]

query = """
    SELECT DISTINCT f1.defence_name
    FROM (
        SELECT f.defence_name
        FROM (
            SELECT c.name_embedding, d.name AS defence_name, d.appropriateatdemurerstage, 
                cosine_similarity(%s::double precision[], c.name_embedding) as Distance1
            FROM public.causes_of_action AS c
            INNER JOIN defenses d
                ON c.id=d.cause_of_action_id
        ) AS f
        WHERE f.appropriateatdemurerstage > '7' AND f.Distance1 > 0.7
        ORDER BY f.Distance1 DESC
    ) as F1
"""

conn = psycopg2.connect(CONN_STRING)
cursor = conn.cursor()

cursor.execute(query, (input_embedding_coa_name,))
cosine_similarity = cursor.fetchall()
cursor.close()
conn.close()
cosine_similarity